*[본 포스팅은 김성훈 교수님의 PyTorch Zero To All Lecture을 보며 작성한 글입니다]*

## 들어가기 전에...

앞선 포스팅에서는 Output size가 1인 경우에 대해서만 다뤘다. 그러나 우리 주변에는 Output size가 그 이상인 경우가 훨씬 많다. 아주 basic한 dataset으로 불려지는 MNIST 데이터셋 또한 0부터 9까지의 10가지 label을 갖고 있어 10개의 Output을 필요로 한다. 이 경우 우리는 Activation Function으로 Softmax함수를 활용하게 된다.

Softmax 함수는 다음과 같이 구성된다.

### Softmax 함수

$$
p_j=\frac{e^{z_j}}{\displaystyle\sum^K_{k=1}e^{z_j}}\\j = 1, 2, ..., K
$$

Softmax 함수의 작동 원리를 설명하는 것은 다소 길어질 수 있어 따로 정리하도록 하겠다.

중요한 점은 Softmax Function의 결과는 원하는 Output 결과 갯수만큼의 0과 1 사이의 값으로 떨어지게 되고, 이 값들의 합은 1이 된다.

Multi-class classification 문제를 해결하기 위해 softmax 함수와 함께 사용하는 Loss Function은 Cross Entropy Loss이다.



## Import Library

In [1]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

In [2]:
import torch
torch.__version__

'1.9.0'

In [3]:
# Training settings
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

Training MNIST Model on cuda


In [11]:
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                               batch_size=batch_size,
                               shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                              batch_size=batch_size,
                              shuffle=False)

2.3%

64.5%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100.0%


Extracting ./mnist_data/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist_data/MNIST\raw



102.8%

Extracting ./mnist_data/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist_data/MNIST\raw




28.6%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

In [13]:
model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [14]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

In [15]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.296072
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 2.295131
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 2.301317
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 2.314219
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2.298035
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 2.304544
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 2.287348
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 2.293203
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 2.300339
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 2.306266
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 2.291584
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 2.287301
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 2.292174
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 2.284783
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 2.301886
Train Epoch: 1 | Batch Status: 9600/60